In [2]:
import cabi_functions
import pandas as pd

# Data loading
First we load the table of all trips, remove any with incomplete data, and convert the datatypes to the most appropriate

In [7]:
df = cabi_functions.return_trip_datatable()
# clean the NA values out
df = df.dropna()
# convert them to appropriate datatypes
df = df.convert_dtypes()


We have some number of trips where the end station is listed as 0. Obviously these are invalid

In [9]:
df = df[df.start_station_id>0]
df = df[df.end_station_id>0]


<ipython-input-9-41854cb5f6ae>:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  df = df[df.start_station_id in station_names.values]


ValueError: ('Lengths must match to compare', (28556925,), (652,))

We also need to make sure we know the location of the station. Therefore we load the list of stations names

In [15]:
# Define which attributes to lookup from airports.csv
cabi_stations = 'https://raw.githubusercontent.com/mlinds/cabi-data/main/data/stationLookup.csv'
station_names_list = list(pd.read_csv(cabi_stations).short_name)

We rewrite the dataframe to include only stations that exist in our location lookup table

In [17]:
df = df[df.end_station_id.map(lambda x:x in station_names_list)]
df = df[df.start_station_id.map(lambda x:x in station_names_list)]

In [19]:
# create a list of the sorted stations
sorted_stations = [sorted([int(x),int(y)]) for x,y in zip(df.start_station_id,df.end_station_id)]
sorted_stations_combined = [int(str(x)+str(y)) for x,y in sorted_stations]
# assign the station to a column in the dataframe,and group it by the unique station combo, then return the results to a seperate dataframe
grouped = df.assign(sorted_stations=sorted_stations_combined).groupby('sorted_stations')

route_popularity = grouped.count().reset_index()
route_popularity = route_popularity[['sorted_stations','end_station_id']]
route_popularity.columns=['sorted','popularity']

In [ ]:
a = [int(str(val)[0:5]) for val in route_popularity.sorted]
b = [int(str(val)[5:10]) for val in route_popularity.sorted]

route_popularity = route_popularity.assign(st=a,en=b)

In [ ]:
route_popularity.to_csv('data/connections_csv.csv',columns=['st','en','popularity'],index=False)